# Imports

In [1]:
import torch 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [2]:
if device.type == 'cpu':
    # CPU version
    ! pip install -q torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric --no-index --find-links=file:///kaggle/input/pytorch-geometric/PyTorch-Geometric
elif device.type == 'cuda':
    # GPU version
    ! pip install -q torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric --no-index --find-links=file:///kaggle/input/pytorchgeometric
else:
    raise Exception('Bruh.')

In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc

from torch_geometric.loader import DataLoader

# Add python files
import sys
sys.path.append('/kaggle/input/icecube-py')
from dataset import MyOwnDataset
from metrics import angular_dist_score
import pred_to_angles

# Load a pre-trained model

In [4]:
import torch
from torch.nn import Linear, LeakyReLU
from torch_geometric.nn import DynamicEdgeConv
from torch_geometric.nn import global_mean_pool
from torch import Tensor, LongTensor
from torch_scatter import scatter_mean
from torch_geometric.utils.homophily import homophily
from torch_geometric.nn.aggr import MultiAggregation, AttentionalAggregation


class EdgeConvMLP(torch.nn.Module):
    """Basic convolutional block."""
    def __init__(self, dim_in, dim_hidden, dim_out):
        super().__init__()

        self.sequential = torch.nn.Sequential(
            Linear(dim_in, dim_hidden),
            LeakyReLU(),
            Linear(dim_hidden, dim_out),
            LeakyReLU(),
        )

    def forward(self, x):
        return self.sequential(x)
    
class GateMLP(torch.nn.Module):
    """Basic convolutional block."""
    def __init__(self, dim_in, dim_hidden, dim_out):
        super().__init__()

        self.sequential = torch.nn.Sequential(
            Linear(dim_in, dim_hidden),
            LeakyReLU(),
            Linear(dim_hidden, dim_out),
            LeakyReLU(),
        )

    def forward(self, x):
        return self.sequential(x)


class DynEdgeAttention(torch.nn.Module):
    """Dynedge model from https://iopscience.iop.org/article/10.1088/1748-0221/17/11/P11003)"""
    def __init__(self, num_node_features, dim_output, dropout_rate=0.):
        super(DynEdgeAttention, self).__init__()
        
        torch.manual_seed(12345)
        self.num_node_features = num_node_features
        self.dim_output = dim_output
        self.dropout_rate = dropout_rate
        self.K = 8
#         self.aggrs_list = ['mean', 'min' , 'max', 'sum', AttentionalAggregation(gate_nn=GateMLP(256, 512, 256))]
        self.aggrs_list = [AttentionalAggregation(gate_nn=GateMLP(256, 64, 1))]

        self.conv1 = DynamicEdgeConv(nn=EdgeConvMLP(2 * self.num_node_features, 336, 256), k=self.K)
        self.conv2 = DynamicEdgeConv(nn=EdgeConvMLP(512, 336, 256), k=self.K)
        self.conv3 = DynamicEdgeConv(nn=EdgeConvMLP(512, 336, 256), k=self.K)
        self.conv4 = DynamicEdgeConv(nn=EdgeConvMLP(512, 336, 256), k=self.K)
        
        # final regressor
        self.mlp1 = torch.nn.Sequential(
            Linear(256 * 4 + self.num_node_features, 336),
            LeakyReLU(),
            Linear(336, 256),
            LeakyReLU(),
        )
        
        self.global_pool =  MultiAggregation(aggrs=self.aggrs_list)
        
#         mode_kwargs = {'in_channels': 256, 'out_channels': 256, 'num_heads': 16}
#         self.global_pool =  MultiAggregation(aggrs=self.aggrs_list, mode='attn', mode_kwargs=mode_kwargs)

#         AttentionalAggregation
#         self.attentional_aggr = AttentionalAggregation(gate_nn=GateMLP(256, 512, 256))

        self.mlp2 =  torch.nn.Sequential(
            Linear(len(self.aggrs_list) * 256 + (4 + self.num_node_features), 128), # input depends of number of aggregating fns + 4 homophily + mean_node
#             Linear(256 + (4 + self.num_node_features), 128),
            LeakyReLU(),
            Linear(128, self.dim_output)
        )


    def _calculate_global_variables(
        self,
        x: Tensor,
        edge_index: LongTensor,
        batch: LongTensor,
    ) -> Tensor:
        """Calculate global variables."""
        # Calculate homophily (scalar variables)
        h_x = homophily(edge_index, x[:, 0], batch).reshape(-1, 1)
        h_y = homophily(edge_index, x[:, 1], batch).reshape(-1, 1)
        h_z = homophily(edge_index, x[:, 2], batch).reshape(-1, 1)
        h_t = homophily(edge_index, x[:, 3], batch).reshape(-1, 1)
        
        # Calculate mean features
        global_means = scatter_mean(x, batch, dim=0)

        # Add global variables
        global_variables = torch.cat([global_means, h_x, h_y, h_z, h_t], dim=-1)

        return global_variables

    def forward(self, x, edge_index, batch):
        # 0. Obtain global variables
        global_x = self._calculate_global_variables(x, edge_index, batch)
        
        # 1. Obtain node embeddings at various embedding depths
        x1 = self.conv1(x, batch)
        x2 = self.conv2(x1, batch)
        x3 = self.conv3(x2, batch)
        x4 = self.conv4(x3, batch)

        x = torch.cat([x, x1, x2, x3, x4], dim=-1)
        
        x = self.mlp1(x)

        # 2. Pooling        
        x = self.global_pool(x, batch)
            
        x = torch.cat([global_x, x], dim=-1)

        # 3. Apply a final MLP regressor
        x = self.mlp2(x)
        
        return x

In [5]:
# Initalize your final model
model = DynEdgeAttention(
    num_node_features=5, 
    dim_output=3, 
    dropout_rate=0.
).to(device)

# Load model from path
PATH_LOAD = '/kaggle/input/icecube-models/26-02-dynedgeattentionxyz-0to41-expected1.075.pt'

target_mode = 'xyz' # angles / cossin / xyz


if device.type == 'cpu':
    model.load_state_dict(torch.load(PATH_LOAD, map_location=torch.device('cpu')))
else: # GPU - cuda
    model.load_state_dict(torch.load(PATH_LOAD))
    
model.eval()

DynEdgeAttention(
  (conv1): DynamicEdgeConv(nn=EdgeConvMLP(
    (sequential): Sequential(
      (0): Linear(in_features=10, out_features=336, bias=True)
      (1): LeakyReLU(negative_slope=0.01)
      (2): Linear(in_features=336, out_features=256, bias=True)
      (3): LeakyReLU(negative_slope=0.01)
    )
  ), k=8)
  (conv2): DynamicEdgeConv(nn=EdgeConvMLP(
    (sequential): Sequential(
      (0): Linear(in_features=512, out_features=336, bias=True)
      (1): LeakyReLU(negative_slope=0.01)
      (2): Linear(in_features=336, out_features=256, bias=True)
      (3): LeakyReLU(negative_slope=0.01)
    )
  ), k=8)
  (conv3): DynamicEdgeConv(nn=EdgeConvMLP(
    (sequential): Sequential(
      (0): Linear(in_features=512, out_features=336, bias=True)
      (1): LeakyReLU(negative_slope=0.01)
      (2): Linear(in_features=336, out_features=256, bias=True)
      (3): LeakyReLU(negative_slope=0.01)
    )
  ), k=8)
  (conv4): DynamicEdgeConv(nn=EdgeConvMLP(
    (sequential): Sequential(
      (

# Data Loading

In [6]:
%%time
test_meta_df = pd.read_parquet('/kaggle/input/icecube-neutrinos-in-deep-ice/test_meta.parquet')
# test_meta_df = pd.read_parquet('/kaggle/input/smallermeta/val_meta_11_small.parquet')

test_meta_df

CPU times: user 15.2 ms, sys: 6.21 ms, total: 21.5 ms
Wall time: 74.8 ms


,batch_id,event_id,first_pulse_index,last_pulse_index
0,661,2092,0,298
1,661,7344,299,334
2,661,9482,335,377


In [7]:
test_batch_ids = test_meta_df.batch_id.unique()
test_batch_ids

array([661])

# Prediction loop

In [8]:
bs_pred = 100 # batchsize for predictions
metrics = False

In [9]:
%%time
list_azs = []
list_zens = []
list_event_ids = []

for batch_id in test_batch_ids:
    
    print(f'=========== START PREDICTIONS BATCH {batch_id} ===========')
        
    dataset = MyOwnDataset(
        batch_id, 
#         path_batch=f'/kaggle/input/smallermeta/batch_11_small.parquet',
        path_batch=f'/kaggle/input/icecube-neutrinos-in-deep-ice/test/batch_{batch_id}.parquet',
#         path_meta='/kaggle/input/smallermeta/val_meta_11_small.parquet',         
        path_meta='/kaggle/input/icecube-neutrinos-in-deep-ice/test_meta.parquet', 
        path_sensor='/kaggle/input/icecube-neutrinos-in-deep-ice/sensor_geometry.csv',
        target_mode=target_mode, 
        K=8, 
        features=['x', 'y', 'z', 'time', 'charge'], 
        threshold_events=500,
        targets_test=True
    )     

    data_loader = DataLoader(dataset, batch_size=bs_pred, shuffle=False)
    
    angle_error_sum = 0
    
    for id_batch, data in enumerate(tqdm(data_loader)): # Iterate over batches 
        
        # Load data and labels to device and predict
        events, event_ids = data
        x, edge_index, batch = events.x.to(device), events.edge_index.to(device), events.batch.to(device)
        
        # for big events, do not compute
        labels = events.y.to(device).reshape(-1, model.dim_output) # reshape bc model returns (batchsize, dim_out), while loader (idiot!) returns (dim_out*batchsize)

        out = model(x, edge_index, batch) # Perform a single forward pass

        # Convert preds to angles - same for labels 
        if target_mode == 'angles':
            az_true, zen_true, az_pred, zen_pred = pred_to_angles.from_angles(out, labels)
        if target_mode == 'cossin':        
            az_true, zen_true, az_pred, zen_pred = pred_to_angles.from_cossin(out, labels)
        if target_mode == 'xyz':
            az_true, zen_true, az_pred, zen_pred = pred_to_angles.from_xyz(out, labels)

        # Detach from GPU and send to CPU - convert to np to be accepted by host metric function
        az_pred = az_pred.detach().cpu().numpy()
        zen_pred = zen_pred.detach().cpu().numpy()
        az_true = az_true.detach().cpu().numpy()
        zen_true = zen_true.detach().cpu().numpy()
            
        # Metrics
        if metrics:
            angle_error = angular_dist_score(az_true, zen_true, az_pred, zen_pred)
            angle_error_sum += angle_error * events.num_graphs

            if id_batch % 100 == 0:
                print(f'Batch {id_batch}/{len(data_loader)} - Angle error {angle_error}') 
        
        list_event_ids.append(event_ids)
        list_azs.append(az_pred)
        list_zens.append(zen_pred)

    if metrics:
        print(angle_error_sum / len(data_loader.dataset))
    
    del dataset, data_loader
    gc.collect()

=========== START PREDICTIONS BATCH 661 ===========
Preparing batch 661 ...
Loading meta ...
Loading sensor ...
Loading batch ...
Complete.



100%|██████████| 1/1 [00:01<00:00,  1.03s/it]

CPU times: user 489 ms, sys: 127 ms, total: 616 ms
Wall time: 1.22 s


# Convert preds to csv

In [10]:
submission = pd.DataFrame(
    {
        'event_id': np.concatenate(list_event_ids, axis=0),  
        'azimuth': np.concatenate(list_azs, axis=0),  
        'zenith': np.concatenate(list_zens, axis=0)
    }
)
submission

,event_id,azimuth,zenith
0,2092,-0.319548,0.757679
1,7344,-0.103190,1.205279
2,9482,-1.918516,1.651994


In [11]:
submission.to_csv('submission.csv', index=False)